<a href="https://colab.research.google.com/github/google-research/tapas/blob/master/notebooks/sqa_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2020 The Google AI Language Team Authors

Licensed under the Apache License, Version 2.0 (the "License");

In [1]:
# Copyright 2019 The Google AI Language Team Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Running a Tapas fine-tuned checkpoint
---
This notebook shows how to load and make predictions with TAPAS model, which was introduced in the paper: [TAPAS: Weakly Supervised Table Parsing via Pre-training](https://arxiv.org/abs/2004.02349)

# Clone and install the repository


First, let's fetch the code from the github repository and install it

In [2]:
! git clone https://github.com/google-research/tapas.git

Cloning into 'tapas'...
remote: Enumerating objects: 442, done.
remote: Total 442 (delta 0), reused 0 (delta 0), pack-reused 442
Receiving objects: 100% (442/442), 298.94 KiB | 14.95 MiB/s, done.
Resolving deltas: 100% (276/276), done.


In [3]:
! pip install ./tapas

Processing ./tapas
     |████████████████████████████████| 3.5MB 13.0MB/s 
     |████████████████████████████████| 10.1MB 36.1MB/s 
     |████████████████████████████████| 516.2MB 31kB/s 
     |████████████████████████████████| 716kB 27kB/s 
     |████████████████████████████████| 61kB 6.2MB/s 
     |████████████████████████████████| 3.5MB 10.9MB/s 
     |████████████████████████████████| 358kB 13.7MB/s 
     |████████████████████████████████| 1.4MB 12.7MB/s 
     |████████████████████████████████| 153kB 14.9MB/s 
     |████████████████████████████████| 61kB 8.4MB/s 
     |████████████████████████████████| 225kB 13.9MB/s 
     |████████████████████████████████| 63.2MB 50kB/s 
     |████████████████████████████████| 51kB 8.8MB/s 
     |████████████████████████████████| 81kB 10.3MB/s 
     |████████████████████████████████| 1.2MB 29.5MB/s 
     |████████████████████████████████| 215kB 32.6MB/s 
     |████████████████████████████████| 184kB 49.9MB/s 
     |████████████████████████████████

# Fetch models fom Google Storage

Next we can get pretrained checkpoint from Google Storage. For the sake of speed, this is base sized model trained on [SQA](https://www.microsoft.com/en-us/download/details.aspx?id=54253). Note that best results in the paper were obtained with with a large model, with 24 layers instead of 12.

In [4]:
! gsutil cp gs://tapas_models/2020_04_21/tapas_sqa_base.zip . && unzip tapas_sqa_base.zip

Copying gs://tapas_models/2020_04_21/tapas_sqa_base.zip...
/ [1 files][  1.0 GiB/  1.0 GiB]   65.4 MiB/s                                   
Operation completed over 1 objects/1.0 GiB.                                      
Archive:  tapas_sqa_base.zip
   creating: tapas_sqa_base/
  inflating: tapas_sqa_base/model.ckpt.data-00000-of-00001  
  inflating: tapas_sqa_base/model.ckpt.index  
  inflating: tapas_sqa_base/README.txt  
  inflating: tapas_sqa_base/vocab.txt  
  inflating: tapas_sqa_base/bert_config.json  
  inflating: tapas_sqa_base/model.ckpt.meta  


# Imports

In [5]:
import tensorflow.compat.v1 as tf
import os 
import shutil
import csv
import pandas as pd
import IPython

tf.get_logger().setLevel('ERROR')

In [6]:
from tapas.utils import tf_example_utils
from tapas.protos import interaction_pb2
from tapas.utils import number_annotation_utils
from tapas.scripts import prediction_utils

# Load checkpoint for prediction

Here's the prediction code, which will create and `interaction_pb2.Interaction` protobuf object, which is the datastructure we use to store examples, and then call the prediction script.

In [7]:
os.makedirs('results/sqa/tf_examples', exist_ok=True)
os.makedirs('results/sqa/model', exist_ok=True)
with open('results/sqa/model/checkpoint', 'w') as f:
  f.write('model_checkpoint_path: "model.ckpt-0"')
for suffix in ['.data-00000-of-00001', '.index', '.meta']:
  shutil.copyfile(f'tapas_sqa_base/model.ckpt{suffix}', f'results/sqa/model/model.ckpt-0{suffix}')

In [18]:
#Giving URL to my Github profile
url = 'https://github.com/ifaizankhan/Resume-Query-using-BERT-GoogleTapas.git'

#Converting raw version of the csv file into python dataframe 
df = pd.read_csv('https://github.com/ifaizankhan/Resume-Query-using-BERT-GoogleTapas/raw/main/Resume_Parsing/data.csv')


In [19]:
df = df.astype(str)

In [20]:
df.head(10)

,name,email,mobile_number,skills,college_name,degree,designation,experience,company_names,no_of_pages,total_experience
0,Name:,abiral.pandey88@gmail.com,940-242-3303,"['Technical skills', 'Pl/sql', 'Mysql', 'Troub...",nan,nan,nan,[],"['oracle', 'Microsoft', 'Oracle']",nan,0.0
1,Amar Amar.srbsa@gmail.com,Amar.srbsa@gmail.com,201-708-8565,"['Merchant', 'Css', 'Analyze', 'Presentation',...",nan,['Bachelors in Computer Science & Technology'],['Business Analyst Online Banking'],nan,"['Amazon', 'Oracle']",nan,0.0
2,Avinash G,Avinash87.java@gmail.com,913-730-0694,"['Css', 'International', 'Analyze', 'Presentat...",nan,"['Degree: Bachelor of Technology, Electronics ...",nan,nan,['Oracle'],nan,0.0
3,Ashok Jayakumar,Venkat@infoservicesllc.com,734-259-2421,"['Security', 'Daily operations', 'Zookeeper', ...",nan,nan,nan,nan,nan,nan,0.0
4,Chandler Robert Durairaj Joshua Business,chandler.neel@gmail.com,515-257-3838,"['Prototype', 'Ms project', 'Pl/sql', 'Mysql',...",nan,nan,"['Customer Support Executive', 'Project Manage...",nan,['Microsoft'],nan,0.0
5,BHARAT ARORA,nan,732-917-4097,"['Financial statements', 'Policies', 'Erp', 'T...",nan,nan,nan,nan,['Oracle'],nan,0.0
6,NAVNEET 407-,11baconsultant@outlook.com,407-401-9023,"['Cms', 'Recruitment', 'Analyze', 'Scheduling'...",nan,['Degree Area of Study University Location MS...,nan,nan,nan,nan,0.0
7,KIRAN Business Analyst,nan,nan,"['Risk assessment', 'Analyze', 'Sap', 'Finance...",nan,nan,nan,nan,['Oracle'],nan,0.0
8,Anil Krishna Mogalaturthi,Krish.java23@gmail.com,681-888-2999,"['Profiling', 'Contracts', 'Css', 'Payroll', '...",nan,nan,nan,nan,"['oracle', 'Amazon', 'Oracle']",nan,0.0
9,Anil Agarwal,Email:ramakrishna@infoservicesllc.com,609 772 9034,"['Risk assessment', 'Policies', 'Pl/sql', 'Var...",nan,nan,"['Process Consultant', 'Project Manager', 'Bus...",nan,"['Latin America', 'Knight Merz', 'Microsoft', ...",nan,0.0


In [21]:
list_of_list = [[]]
list_of_list[0] = list(df.columns)
list_of_list.extend(df.values.tolist())

In [23]:
max_seq_length = 512
vocab_file = "tapas_sqa_base/vocab.txt"
config = tf_example_utils.ClassifierConversionConfig(
    vocab_file=vocab_file,
    max_seq_length=max_seq_length,
    max_column_id=max_seq_length,
    max_row_id=max_seq_length,
    strip_column_names=False,
    add_aggregation_candidates=False,
)
converter = tf_example_utils.ToClassifierTensorflowExample(config)

def convert_interactions_to_examples(tables_and_queries):
  """Calls Tapas converter to convert interaction to example."""
  for idx, (table, queries) in enumerate(tables_and_queries):
    interaction = interaction_pb2.Interaction()
    for position, query in enumerate(queries):
      question = interaction.questions.add()
      question.original_text = query
      question.id = f"{idx}-0_{position}"
    for header in table[0]:
      interaction.table.columns.add().text = header
    for line in table[1:]:
      row = interaction.table.rows.add()
      for cell in line:
        row.cells.add().text = cell
    number_annotation_utils.add_numeric_values(interaction)
    for i in range(len(interaction.questions)):
      try:
        yield converter.convert(interaction, i)
      except ValueError as e:
        print(f"Can't convert interaction: {interaction.id} error: {e}")
        
def write_tf_example(filename, examples):
  with tf.io.TFRecordWriter(filename) as writer:
    for example in examples:
      writer.write(example.SerializeToString())

def predict(table_data, queries):
  table = table_data
  examples = convert_interactions_to_examples([(table, queries)])
  write_tf_example("results/sqa/tf_examples/test.tfrecord", examples)
  write_tf_example("results/sqa/tf_examples/random-split-1-dev.tfrecord", [])
  
  ! python tapas/tapas/run_task_main.py \
    --task="SQA" \
    --output_dir="results" \
    --noloop_predict \
    --test_batch_size={len(queries)} \
    --tapas_verbosity="ERROR" \
    --compression_type= \
    --init_checkpoint="tapas_sqa_base/model.ckpt" \
    --bert_config_file="tapas_sqa_base/bert_config.json" \
    --mode="predict" 2> error


  results_path = "results/sqa/model/test_sequence.tsv"
  all_coordinates = []
  df = pd.DataFrame(table[1:], columns=table[0])
  display(IPython.display.HTML(df.to_html(index=False)))
  print()
  with open(results_path) as csvfile:
    reader = csv.DictReader(csvfile, delimiter='\t')
    for row in reader:
      coordinates = prediction_utils.parse_coordinates(row["answer_coordinates"])
      all_coordinates.append(coordinates)
      answers = ', '.join([table[row + 1][col] for row, col in coordinates])
      position = int(row['position'])
      print(">", queries[position])
      print(answers)
  return all_coordinates

# Predict

In [24]:
result = predict(list_of_list, ["what were the candidate names?"])

is_built_with_cuda: True
is_gpu_available: True
GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Training or predicting ...
Evaluation finished after training step 0.


name,email,mobile_number,skills,college_name,degree,designation,experience,company_names,no_of_pages,total_experience
Name:,abiral.pandey88@gmail.com,940-242-3303,"['Technical skills', 'Pl/sql', 'Mysql', 'Troubleshooting', 'Nosql', 'E-commerce', 'Security', 'Css', 'Programming', 'Design', 'Email', 'Agile', 'Sql', 'Process', 'Db2', 'System', 'Architecture', 'Sdlc', 'Documentation', 'P', 'Writing', 'Testing', 'Html', 'Jira', 'Installation', 'Operating systems', 'Analysis', 'Xml', 'Servers', 'Oracle', 'Logging', 'Windows', 'Database', 'Solaris', 'Ibm', 'Ux', 'Html5', 'Ui', 'Unix', 'Shell', 'Pattern', 'Aws', 'Interactive', 'Debugging', 'Api', 'Js', 'Javascript', 'Cloud', 'Access', 'Scripting', 'Reports', 'Rest', 'Requests', 'Technical', 'Automation', 'Excel', 'Etl', 'Queries', 'Linux', 'Java', 'Github', 'Reporting', 'Web services', 'Jsp', 'Visio', 'Postgresql', 'Soap', 'Computer science', 'Test cases']",nan,nan,nan,[],"['oracle', 'Microsoft', 'Oracle']",nan,0.0
Amar Amar.srbsa@gmail.com,Amar.srbsa@gmail.com,201-708-8565,"['Merchant', 'Css', 'Analyze', 'Presentation', 'Data analysis', 'Sharepoint', 'Operations', 'Payments', 'Quality assurance', 'Ibm', 'Html5', 'Ui', 'Saas', 'Specifications', 'Lifecycle', 'Cloud', 'Tfs', 'Mortgage', 'Reports', 'Tableau', 'Transactions', 'Microsoft visio', 'Jsp', 'Budget', 'Modeling', 'Test cases', 'Ms project', 'Regulatory', 'Powerpoint', 'Sql server', 'Sql', 'System', 'Db2', 'Architecture', 'Retail', 'Conversion', 'Word', 'Communication', 'Scrum', 'Windows', 'Project management', 'Access', 'Test plans', 'Usability', 'Reporting', 'Deposits', 'Soap', 'Hp alm', 'Pl/sql', 'Administration', 'User stories', 'Agile', 'Documentation', 'Ms excel', 'Jira', 'Html', 'Business process', 'Gap analysis', 'Warehouse', 'Oracle', 'Ux', 'Key performance indicators', 'Pivot', 'Drafting', 'Api', 'Visual', 'Strategy', 'Automation', 'Queries', 'Real-time', 'Java', 'Plan', 'Visio', 'Computer science', 'Kpis', 'Security', 'Programming', 'Design', 'Regulations', 'Process', 'Sdlc', 'Writing', 'Testing', 'Apis', 'Jad', 'Analysis', 'Xml', 'Compliance', 'Database', 'Aws', 'Mock', 'Workflow', 'Analytical', 'Javascript', 'Js', 'Matrix', 'Microsoft office', 'Technical', 'Selenium', 'Excel', 'Etl', 'Swot analysis', 'Banking', 'Financial reporting', 'Underwriting']",nan,['Bachelors in Computer Science & Technology'],['Business Analyst Online Banking'],nan,"['Amazon', 'Oracle']",nan,0.0
Avinash G,Avinash87.java@gmail.com,913-730-0694,"['Css', 'International', 'Analyze', 'Presentation', 'Json', 'Operations', 'Content', 'Threading', 'Analytics', 'Ibm', 'Html5', 'Ui', 'Pattern', 'Debugging', 'Cloud', 'Scripting', 'Reports', 'Rest', 'Requests', 'Jsp', 'Modeling', 'Test cases', 'Networking', 'Mysql', 'Responses', 'Sql server', 'Inventory', 'Editing', 'Sql', 'Db2', 'System', 'Architecture', 'Mobile', 'Mining', 'Retail', 'Communication', 'Scrum', 'Windows', 'Unix', 'Access', 'Linux', 'Web services', 'Reporting', 'Postgresql', 'Docker', 'Soap', 'Prototype', 'Pl/sql', 'Redhat', 'User stories', 'Agile', 'Html', 'Jira', 'Oracle', 'Ux', 'Shell', 'Api', 'Strategy', 'Queries', 'Publishing', 'Java', 'Github', 'Controls', 'Plan', 'Visio', 'Technical skills', 'Security', 'Programming', 'Design', 'Process', 'Transportation', 'Website', 'Writing', 'Testing', 'Electronics', 'Apis', 'Analysis', 'Xml', 'Servers', 'Logging', 'Database', 'Aws', 'Mock', 'Interactive', 'Workflow', 'Javascript', 'Js', 'Intranet', 'Engineering', 'Selenium', 'Technical', 'Legal', 'Banking']",nan,"['Degree: Bachelor of Technology, Electronics and Communication Engineering']",nan,nan,['Oracle'],nan,0.0
Ashok Jayakumar,Venkat@infoservicesllc.com,734-259-2421,"['Security', 'Daily operations', 'Zookeeper', '.net', 'Sales operations', 'Hbase', 'Brand', 'Analyze', 'Logistics', 'Administration', 'Design', 'Agile', 'Sql', 'Website', 'System', 'Db2', 'Process', 'Audit', 'Hive', 'Mobile', 'Documentation', 'Marketing', 'Sharepoint', 'Testing', 'Html', 'Certification', 'Improvement', 'Operat


> what were the candidate names?
[Type text, Ashok Jayakumar, Anil Krishna Mogalaturthi, Adhi Gopalam adhigopalam@gmail.com, Bharatha jyothi Bommireddy, Anudeep Sr Java Programmer anudeepreddynallamada@gmail.com Professional, AMULYA KOMATINENI, Chandler Robert Durairaj Joshua Business, Anil Agarwal, Name: Balakrishna Sudabathula, Abhishek Bhushan, GOPI SID Email, Suresh Basetti, BHARAT ARORA, Alekhya (, Email ID, ATUL MATHUR, Akhil                                                                                , NAVNEET  407-, Amrinder Pelia, Akhil                                                                                , Amar Amar.srbsa@gmail.com, Bapuji Sr, Ami Jape 50 GRAHAM, Achyuth 540, KIRAN Business Analyst, E-mail, B Shaker, Avinash G, Hyma Gumpu, Avanthi Katta Sr. Business Analyst, Ajay Kumar, Adelina Erimia
